### Full d20 pretrain

I'm now going to try the full d20 pretrain on an 8xH100 machine on lambda cloud.

First I'll follow the instructions in `trying-lambda-cloud.ipynb` to get ready.

If I'm able to get a machine in the same region as my test (us-south-2) then I can use the same storage and the tokenizer should already be there. If not, I'll scp the tokenizer from my mac:

```
cd tokenizer-from-challenge-25
scp my-tokenizer.pkl ubuntu@[ip]:/home/ubuntu/mynanochat/
scp token_bytes.pt ubuntu@[ip]:/home/ubuntu/mynanochat/
```

Then I'll do this:

In tmux shell:

```
source .venv/bin/activate

cd challenge-25-pretrain-d20

export PYTHONPATH=../my_nanochat/

torchrun --standalone --nproc_per_node=1 -m scripts.my_base_train -- --depth=20 --run=challenge-25-3 > train_output_003.txt 2>&1
```